# Deep Research Agent

## Imports

In [61]:
import os
import openai
import json
from dataclasses import dataclass, field
from typing import List
from tavily import TavilyClient
from json.decoder import JSONDecodeError
from pydantic_settings import BaseSettings
from IPython.display import Markdown

import time


### Constructing application Configuration object

Note that we have two LLMs configures, we will be using a reasoning model (DeepSeek-R1) for some of the sub agents while we will be using a regular instruction tuned Llama model (Meta-Llama-3.3-70B-Instruct) for other.

In [62]:
class Config(BaseSettings):
    SAMBANOVA_API_KEY: str
    SAMBANOVA_BASE_URL: str
    LLM_REASONING: str
    LLM_REGULAR: str
    TAVILY_API_KEY: str

Be sure to have your SAMBANOVA_API_KEY (get it [here](https://fnf.dev/4aVUqro)) and TAVILY_API_KEY (get it [here](https://app.tavily.com/)) exported as environment variables before running the next cell.

In [63]:
config = Config(SAMBANOVA_API_KEY=os.environ["SAMBANOVA_API_KEY"],
                SAMBANOVA_BASE_URL="https://api.sambanova.ai/v1",
                LLM_REASONING="DeepSeek-R1-Distill-Llama-70B",
                LLM_REGULAR="Meta-Llama-3.3-70B-Instruct",
                TAVILY_API_KEY=os.environ["TAVILY_API_KEY"])

### Data Classes to define the System State

In [64]:
@dataclass
class Search:
    url: str = ""
    content: str = ""

@dataclass
class Research:
    search_history: List[Search] = field(default_factory=list)
    latest_summary: str = ""
    reflection_iteration: int = 0

@dataclass
class Paragraph:
    title: str = ""
    content: str = ""
    research: Research = field(default_factory=Research)

@dataclass
class State:
    report_title: str = ""
    paragraphs: List[Paragraph] = field(default_factory=list)

### Helper functions for data cleaning

In [65]:
def remove_reasoning_from_output(output):
    return output.split("</think>")[-1].strip()

def clean_json_tags(text):
    return text.replace("```json\n", "").replace("\n```", "")

def clean_markdown_tags(text):
    return text.replace("```markdown\n", "").replace("\n```", "")

def callOpenAI(self, inputModel, system_prompt, user_prompt):
    for attempt in range(10):
        try:
            response = self.openai_client.chat.completions.create(
                model=inputModel,
                messages=[{"role": "system", "content": system_prompt},
                        {"role":"user","content": user_prompt}]
            )
            break
        except openai.RateLimitError:
            wait = 2 ** attempt
            print(f"Rate limit hit. Retrying in {wait}s...")
            time.sleep(wait)
    
    return response

### Search tool and a fuction to update System State with search results

In [66]:
def tavily_search(query, include_raw_content=True, max_results=3):

    tavily_client = TavilyClient(api_key=config.TAVILY_API_KEY)

    return tavily_client.search(query,
                                include_raw_content=include_raw_content,
                                max_results=max_results)

def update_state_with_search_results(search_results, idx_paragraph, state):
    
    for search_result in search_results["results"]:
        search = Search(url=search_result["url"], content=search_result["raw_content"])
        state.paragraphs[idx_paragraph].research.search_history.append(search)

## Agents

Here we define LLM sub-Agents that will read the System State, perform computation and evolve the state.

### Agent for Report structure creation

In [67]:
output_schema_report_structure = {
        "type": "array",
        "items": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"}
            }
        }
    }

SYSTEM_PROMPT_REPORT_STRUCTURE = f"""
You are a Deep Research assistan. Given a query, plan a structure for a report and the paragraphs to be included.
Make sure that the ordering of paragraphs makes sense.
Once the outline is created, you will be given tools to search the web and reflect for each of the section separately.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_report_structure, indent=2)}
</OUTPUT JSON SCHEMA>

Title and content properties will be used for deeper research.
Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [68]:
class ReportStructureAgent:

    def __init__(self, query: str):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )
        self.query = query

    def run(self) -> str:

        response = callOpenAI(self, config.LLM_REASONING, SYSTEM_PROMPT_REPORT_STRUCTURE, self.query)

        # for attempt in range(5):
        #      try:
        #         response = self.openai_client.chat.completions.create(
        #             model=config.LLM_REASONING,
        #             messages=[{"role": "system", "content": SYSTEM_PROMPT_REPORT_STRUCTURE},
        #                     {"role":"user","content": self.query}]
        #         )
        #         break
        #      except openai.RateLimitError:
        #         wait = 2 ** attempt
        #         print(f"Rate limit hit. Retrying in {wait}s...")
        #         time.sleep(wait)

        # response = self.openai_client.chat.completions.create(
        #     model=config.LLM_REASONING,
        #     messages=[{"role": "system", "content": SYSTEM_PROMPT_REPORT_STRUCTURE},
        #               {"role":"user","content": self.query}]
        # )
        return response.choices[0].message.content

    def mutate_state(self, state: State) -> State:

        report_structure = self.run()
        report_structure = remove_reasoning_from_output(report_structure)
        report_structure = clean_json_tags(report_structure)

        report_structure = json.loads(report_structure)

        for paragraph in report_structure:
            state.paragraphs.append(Paragraph(title=paragraph["title"], content=paragraph["content"]))

        return state

### Agent to figure out the first search query for a given paragraph.

In [69]:
input_schema_first_search = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"}
            }
        }

output_schema_first_search = {
            "type": "object",
            "properties": {
                "search_query": {"type": "string"},
                "reasoning": {"type": "string"}
            }
        }

SYSTEM_PROMPT_FIRST_SEARCH = f"""
You are a Deep Research assistan. You will be given a paragraph in a report, it's title and expected content in the following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_first_search, indent=2)}
</INPUT JSON SCHEMA>

You can use a web search tool that takes a 'search_query' as parameter.
Your job is to reflect on the topic and provide the most optimal web search query to enrich your current knowledge.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_first_search, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [70]:
class FirstSearchAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:
        
        response = callOpenAI(self, config.LLM_REGULAR, SYSTEM_PROMPT_FIRST_SEARCH, message)
        # for attempt in range(5):
        #      try:
        #         response = self.openai_client.chat.completions.create(
        #             model=config.LLM_REGULAR,
        #             messages=[{"role": "system", "content": SYSTEM_PROMPT_FIRST_SEARCH},
        #                     {"role":"user","content": message}]
        #         )
        #         break
        #      except openai.RateLimitError:
        #         wait = 2 ** attempt
        #         print(f"Rate limit hit. Retrying in {wait}s...")
        #         time.sleep(wait)

        # response = self.openai_client.chat.completions.create(
        #     model=config.LLM_REGULAR,
        #     messages=[{"role": "system", "content": SYSTEM_PROMPT_FIRST_SEARCH},
        #               {"role":"user","content": message}]
        # )

        response = remove_reasoning_from_output(response.choices[0].message.content)
        response = clean_json_tags(response)

        response = json.loads(response)

        return response

### Agent to summarise search results of the first search.

In [71]:
input_schema_first_summary = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"},
                "search_query": {"type": "string"},
                "search_results": {
                    "type": "array",
                    "items": {"type": "string"}
                }
            }
        }

output_schema_first_summary = {
            "type": "object",
            "properties": {
                "paragraph_latest_state": {"type": "string"}
            }
        }

SYSTEM_PROMPT_FIRST_SUMMARY = f"""
You are a Deep Research assistan. You will be given a search query, search results and the paragraph a report that you are researching following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_first_summary, indent=2)}
</INPUT JSON SCHEMA>

Your job is to write the paragraph as a researcher using the search results to align with the paragraph topic and structure it properly to be included in the report.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_first_summary, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [72]:
class FirstSummaryAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = callOpenAI(self, config.LLM_REGULAR, SYSTEM_PROMPT_FIRST_SUMMARY, message)

        # for attempt in range(5):
        #      try:
        #         response = self.openai_client.chat.completions.create(
        #             model=config.LLM_REGULAR,
        #             messages=[{"role": "system", "content": SYSTEM_PROMPT_FIRST_SUMMARY},
        #                     {"role":"user","content": message}]
        #         )
        #         break
        #      except openai.RateLimitError:
        #         wait = 2 ** attempt
        #         print(f"Rate limit hit. Retrying in {wait}s...")
        #         time.sleep(wait)


        # response = self.openai_client.chat.completions.create(
        #     model=config.LLM_REGULAR,
        #     messages=[{"role": "system", "content": SYSTEM_PROMPT_FIRST_SUMMARY},
        #               {"role":"user","content": message}]
        # )
        return response.choices[0].message.content

    def mutate_state(self, message: str, idx_paragraph: int, state: State) -> State:

        summary = self.run(message)
        summary = remove_reasoning_from_output(summary)
        summary = clean_json_tags(summary)
        
        try:
            summary = json.loads(summary)
        except JSONDecodeError:
            summary = {"paragraph_latest_state": summary}

        state.paragraphs[idx_paragraph].research.latest_summary = summary["paragraph_latest_state"]

        return state

### Agent to Reflect on the latest state of the paragraph.

In [73]:
input_schema_reflection = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"},
                "paragraph_latest_state": {"type": "string"}
            }
        }

output_schema_reflection = {
            "type": "object",
            "properties": {
                "search_query": {"type": "string"},
                "reasoning": {"type": "string"}
            }
        }

SYSTEM_PROMPT_REFLECTION = f"""
You are a Deep Research assistan. You are responsible for constructing comprehensife paragraphs for a research report. You will be provided paragraph title and planned content summary, also the latest state of the paragraph that you have already created all in the following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_reflection, indent=2)}
</INPUT JSON SCHEMA>

You can use a web search tool that takes a 'search_query' as parameter.
Your job is to reflect on the current state of the paragraph text and think if you havent missed some critical aspect of the topic and provide the most optimal web search query to enrich the latest state.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_reflection, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [74]:
class ReflectionAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = callOpenAI(self, config.LLM_REGULAR, SYSTEM_PROMPT_REFLECTION, message)

        # for attempt in range(5):
        #      try:
        #         response = self.openai_client.chat.completions.create(
        #                             model=config.LLM_REGULAR,
        #                             messages=[{"role": "system", "content": SYSTEM_PROMPT_REFLECTION},
        #               {"role":"user","content": message}]
        #         )
        #         break
        #      except openai.RateLimitError:
        #         wait = 2 ** attempt
        #         print(f"Rate limit hit. Retrying in {wait}s...")
        #         time.sleep(wait)
           
        # response = self.openai_client.chat.completions.create(
        #     model=config.LLM_REGULAR,
        #     messages=[{"role": "system", "content": SYSTEM_PROMPT_REFLECTION},
        #               {"role":"user","content": message}]
        # )

        response = remove_reasoning_from_output(response.choices[0].message.content)
        response = clean_json_tags(response)
        response = json.loads(response)

        return response

### Agent to summarise search results after Reflection.

In [75]:
input_schema_reflection_summary = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"},
                "search_query": {"type": "string"},
                "search_results": {
                    "type": "array",
                    "items": {"type": "string"}
                },
                "paragraph_latest_state": {"type": "string"}
            }
        }

output_schema_reflection_summary = {
            "type": "object",
            "properties": {
                "updated_paragraph_latest_state": {"type": "string"}
            }
        }

SYSTEM_PROMPT_REFLECTION_SUMMARY = f"""
You are a Deep Research assistan.
You will be given a search query, search results, paragraph title and expected content for the paragraph in a report that you are researching.
You are iterating on the paragraph and the latest state of the paragraph is also provided.
The data will be in the following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_reflection_summary, indent=2)}
</INPUT JSON SCHEMA>

Your job is to enrich the current latest state of the paragraph with the search results considering expected content.
Do not remove key information from the latest state and try to enrich it, only add information that is missing.
Structure the paragraph properly to be included in the report.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_reflection_summary, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [76]:
class ReflectionSummaryAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = callOpenAI(self, config.LLM_REGULAR, SYSTEM_PROMPT_REFLECTION_SUMMARY, message)

        # for attempt in range(5):
        #      try:
        #         response = self.openai_client.chat.completions.create(
        #             model=config.LLM_REGULAR,
        #             messages=[{"role": "system", "content": SYSTEM_PROMPT_REFLECTION_SUMMARY},
        #                     {"role":"user","content": message}]
        #         )
        #         break
        #      except openai.error.RateLimitError:
        #         wait = 2 ** attempt
        #         print(f"Rate limit hit. Retrying in {wait}s...")
        #         time.sleep(wait)


        # response = self.openai_client.chat.completions.create(
        #     model=config.LLM_REGULAR,
        #     messages=[{"role": "system", "content": SYSTEM_PROMPT_REFLECTION_SUMMARY},
        #               {"role":"user","content": message}]
        # )
        return response.choices[0].message.content

    def mutate_state(self, message: str, idx_paragraph: int, state: State) -> State:

        summary = self.run(message)
        summary = remove_reasoning_from_output(summary)
        summary = clean_json_tags(summary)

        try:
            summary = json.loads(summary)
        except JSONDecodeError:
            summary = {"updated_paragraph_latest_state": summary}

        state.paragraphs[idx_paragraph].research.latest_summary = summary["updated_paragraph_latest_state"]

        return state

### Agent to summarise results and produce the formatted report

In [77]:
input_schema_report_formatting = {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"},
                    "paragraph_latest_state": {"type": "string"}
            }
        }
    }

SYSTEM_PROMPT_REPORT_FORMATTING = f"""
You are a Deep Research assistan. You have already performed the research and construted final versions of all paragraphs in the report.
You will get the data in the following json format:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_report_formatting, indent=2)}
</INPUT JSON SCHEMA>

Your job is to format the Report nicely and return it in MarkDown.
If Conclusion paragraph is not present, add it to the end of the report from the latest state of the other paragraphs.
Use titles of the paragraphs to create a title for the report.
"""

In [78]:
class ReportFormattingAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = callOpenAI(self, config.LLM_REASONING, SYSTEM_PROMPT_REPORT_FORMATTING, message)

        # for attempt in range(5):
        #      try:
        #         response = self.openai_client.chat.completions.create(
        #             model=config.LLM_REASONING,
        #             messages=[{"role": "system", "content": SYSTEM_PROMPT_REPORT_FORMATTING},
        #                     {"role":"user","content": message}]
        #         )
        #         break
        #      except openai.RateLimitError:
        #         wait = 2 ** attempt
        #         print(f"Rate limit hit. Retrying in {wait}s...")
        #         time.sleep(wait)

        # response = self.openai_client.chat.completions.create(
        #     model=config.LLM_REASONING,
        #     messages=[{"role": "system", "content": SYSTEM_PROMPT_REPORT_FORMATTING},
        #               {"role":"user","content": message}]
        # )

        summary = response.choices[0].message.content
        summary = remove_reasoning_from_output(summary)
        summary = clean_markdown_tags(summary)
        
        return summary

## The Topology of the System

In [79]:
STATE = State()
QUERY="What is the best way to catch trout in the mammoth lakes area?"
NUM_REFLECTIONS = 2
NUM_RESULTS_PER_SEARCH = 3
CAP_SEARCH_LENGTH = 20000

In [80]:
report_structure_agent = ReportStructureAgent(QUERY)

_ = report_structure_agent.mutate_state(STATE)

first_search_agent = FirstSearchAgent()
first_summary_agent = FirstSummaryAgent()
reflection_agent = ReflectionAgent()
reflection_summary_agent = ReflectionSummaryAgent()
report_formatting_agent = ReportFormattingAgent()

print(f"Total Number of Paragraphs: {len(STATE.paragraphs)}")

idx = 1

for paragraph in STATE.paragraphs:

    print(f"\nParagraph {idx}: {paragraph.title}")

    idx += 1


################## Iterate through paragraphs ##################

for j in range(len(STATE.paragraphs)):

    print(f"\n\n==============Paragraph: {j+1}==============\n")
    print(f"=============={STATE.paragraphs[j].title}==============\n")

    ################## First Search ##################
    
    message = json.dumps(
        {
            "title": STATE.paragraphs[j].title, 
            "content": STATE.paragraphs[j].content
        }
    )
    
    output = first_search_agent.run(message)
    
    search_results = tavily_search(output["search_query"], max_results=NUM_RESULTS_PER_SEARCH)
    
    _ = update_state_with_search_results(search_results, j, STATE)
    
    ################## First Search Summary ##################
    
    message = {
        "title": STATE.paragraphs[j].title,
        "content": STATE.paragraphs[j].content,
        "search_query": search_results["query"],
        "search_results": [result["raw_content"][0:CAP_SEARCH_LENGTH] for result in search_results["results"] if result["raw_content"]]
    }
    
    
    _ = first_summary_agent.mutate_state(message=json.dumps(message), idx_paragraph=j, state=STATE)
    
    ################## Run NUM_REFLECTIONS Reflection steps ##################
    
    for i in range(NUM_REFLECTIONS):
    
        print(f"Running reflection: {i+1}")

        ################## Reflection Step ##################
    
        message = {"paragraph_latest_state": STATE.paragraphs[j].research.latest_summary,
                "title": STATE.paragraphs[j].title,
                "content": STATE.paragraphs[j].content}
        
        output = reflection_agent.run(message=json.dumps(message))

        ################## Reflection Search ##################
        
        search_results = tavily_search(output["search_query"])
        
        _ = update_state_with_search_results(search_results, j, STATE)

        ################## Reflection Search Summary ##################
        
        message = {
            "title": STATE.paragraphs[j].title,
            "content": STATE.paragraphs[j].content,
            "search_query": search_results["query"],
            "search_results": [result["raw_content"][0:20000] for result in search_results["results"] if result["raw_content"]],
            "paragraph_latest_state": STATE.paragraphs[j].research.latest_summary
        }
        
        _ = reflection_summary_agent.mutate_state(message=json.dumps(message), idx_paragraph=j, state=STATE)

# ################## Generate Final Report ##################

# report_data = [{"title": paragraph.title, "paragraph_latest_state": paragraph.research.latest_summary} for paragraph in STATE.paragraphs]

# final_report = report_formatting_agent.run(json.dumps(report_data))

Total Number of Paragraphs: 7

Paragraph 1: Introduction to Trout Fishing in Mammoth Lakes

Paragraph 2: Best Locations for Trout Fishing

Paragraph 3: Tackle and Gear Recommendations

Paragraph 4: Seasonal Fishing Patterns

Paragraph 5: Fishing Techniques

Paragraph 6: Local Regulations and Etiquette

Paragraph 7: Seeking Local Advice


==============Paragraph: 1==============

==============Introduction to Trout Fishing in Mammoth Lakes==============

Rate limit hit. Retrying in 1s...
Rate limit hit. Retrying in 2s...
Rate limit hit. Retrying in 4s...
Rate limit hit. Retrying in 8s...
Rate limit hit. Retrying in 16s...
Rate limit hit. Retrying in 32s...
Rate limit hit. Retrying in 64s...
Rate limit hit. Retrying in 128s...


KeyboardInterrupt: 

In [ ]:
################## Generate Final Report ##################

report_data = [{"title": paragraph.title, "paragraph_latest_state": paragraph.research.latest_summary} for paragraph in STATE.paragraphs]
final_report = report_formatting_agent.run(json.dumps(report_data))

In [ ]:
print(STATE.paragraphs[0].title) # Print the latest summary of the first paragraph for verification
print(STATE.paragraphs[1].research.search_history[3]) # Print the latest summary of the first paragraph for verification

allParagraphs = STATE.paragraphs

for a in allParagraphs:
    print(a.title)
    print(a.content)
    print(a.research.latest_summary)
    for b in a.research.search_history:
        print(b.url)

Introduction to Trout Fishing in Mammoth Lakes
Search(url='https://database.reverb.org/fishing-in-mammoth-lakes/', content=None)
Introduction to Trout Fishing in Mammoth Lakes
Mammoth Lakes, located in the Sierra Nevada mountain range, offers excellent trout fishing opportunities. The area's cold, clear waters are home to various trout species, making it a popular destination for anglers.
Mammoth Lakes, located in the Sierra Nevada mountain range, offers excellent trout fishing opportunities. The area's cold, clear waters are home to various trout species, including rainbow, golden, brook, and cutthroat trout. To catch high-mountain trout, anglers must alter their tactics, using small dry flies, nymphs, and streamers for fly anglers, and small Panther Martin or Roostertail spinners and Little Cleo or Kastmaster spoons for spin anglers. It's essential to be stealthy, as high elevation lakes are often clear and calm, making it easier for fish to spot commotion from above. The Kern River,

### Render the final Report

In [ ]:
print(final_report)

# Trout Fishing in Mammoth Lakes: A Comprehensive Guide

## Introduction to Trout Fishing in Mammoth Lakes

Mammoth Lakes, located in the Sierra Nevada mountain range, offers excellent trout fishing opportunities. The area's cold, clear waters are home to various trout species, including rainbow, golden, brook, and cutthroat trout. To catch high-mountain trout, anglers must alter their tactics, using small dry flies, nymphs, and streamers for fly anglers, and small Panther Martin or Roostertail spinners and Little Cleo or Kastmaster spoons for spin anglers. It's essential to be stealthy, as high elevation lakes are often clear and calm, making it easier for fish to spot commotion from above. The Kern River, which flows through the southern Sierra Nevada Mountains, is an incredibly beautiful and productive trout fishing stream, with numerous tributaries and access points, including the Upper Kern River, South Fork Kern River, and Lower Kern River. Anglers can use various techniques, suc

In [ ]:
display(Markdown(final_report))

# Trout Fishing in Mammoth Lakes: A Comprehensive Guide

## Introduction to Trout Fishing in Mammoth Lakes

Mammoth Lakes, located in the Sierra Nevada mountain range, offers excellent trout fishing opportunities. The area's cold, clear waters are home to various trout species, including rainbow, golden, brook, and cutthroat trout. To catch high-mountain trout, anglers must alter their tactics, using small dry flies, nymphs, and streamers for fly anglers, and small Panther Martin or Roostertail spinners and Little Cleo or Kastmaster spoons for spin anglers. It's essential to be stealthy, as high elevation lakes are often clear and calm, making it easier for fish to spot commotion from above. The Kern River, which flows through the southern Sierra Nevada Mountains, is an incredibly beautiful and productive trout fishing stream, with numerous tributaries and access points, including the Upper Kern River, South Fork Kern River, and Lower Kern River. Anglers can use various techniques, such as drift fishing, bait fishing, and casting lures, to catch trout and other species like bass and catfish. Conservation efforts are also crucial, as alpine ecosystems are fragile, and anglers must be considerate of their waste, avoid trampling the land, and pack out their trash to preserve the environment for future generations. Although specific sustainable trout fishing practices in alpine ecosystems were not found, it is generally recommended that anglers follow best practices, such as handling fish gently, avoiding overfishing, and respecting local regulations to ensure the long-term health of trout populations in Mammoth Lakes.

## Best Locations for Trout Fishing

Mammoth Lakes offers some of the best trout fishing in the world, with a variety of locations to choose from, including the Mammoth Lakes Basin, Convict Lake, Crowley Lake, and the San Joaquin River. The area is home to a range of trout species, including rainbow, brook, brown, and golden trout. Anglers can try their hand at various fishing techniques, such as bait casting, spin casting, trolling, fly fishing, and Tenkara. The fishing season in Mono County typically runs from the last Saturday in April to November 15, with some catch-and-release waters remaining open year-round. Before heading out, anglers should check the current fishing regulations, which have been adjusted for the 2024-2025 season, and obtain any necessary licenses. Anglers 16 and older must have a fishing license, which can be purchased annually or for one or two days. It is also essential to be aware of the size limit for trout in Mammoth Lakes, which varies depending on the water and species, and to handle caught fish gently to reduce stress and injury. Additionally, anglers are encouraged to practice catch-and-release fishing to help conserve the trout population and maintain the health of the fishery. The California Department of Fish and Wildlife website provides up-to-date information on fishing regulations, including the new regulations for the 2024-2025 season. With its stunning natural beauty and abundant fish populations, Mammoth Lakes is an ideal destination for both beginner and experienced anglers, offering a range of fishing experiences, from lake fishing to stream fishing, and opportunities to catch trophy-sized trout.

## Tackle and Gear Recommendations

When it comes to tackle and gear for beginners, there are several essential items to consider. A good starting point is a medium action rod with a spinning reel, which can be used for both bait and lure fishing. Monofilament line is a popular choice for beginners, as it is durable and easy to use. In terms of tackle, a beginner's kit should include hooks, weights, and floats. For lures, soft plastics, spinnerbaits, and topwater lures are good options to start with. It's also important to consider the type of fishing you'll be doing and the species you're targeting when choosing your gear. Additionally, a tackle bag or box is necessary for storing and transporting your gear. Some recommended products for beginners include the Favorite Shay Bird Spinning Combo, Ande Braid - Graphite, Mustad Demon Offset Circle Hook, and the Plano Guide Series Tackle System. Furthermore, it's crucial to check local regulations regarding bait usage and restrictions, and to practice responsible fishing by handling fish gently, minimizing airtime, and releasing them quickly. A state fishing license, a fishing rod and reel, 4- to 12-pound-test monofilament fishing line, fishhooks, a plastic or cork bobber, and a selection of live bait or fishing lures are also essential components of a beginner's fishing gear checklist. Moreover, recent advancements in eco-friendly fishing gear, such as biodegradable fishing lines, eco-friendly fishing nets, and sustainable bait options, should be considered to minimize environmental impact. The use of smart fishing technologies, including underwater drones and GPS-enabled tackle, can also enhance the fishing experience while promoting sustainable practices. By investing in these essential items, following local regulations, and adopting responsible and sustainable fishing practices, beginners can set themselves up for success and enjoy a fun and rewarding fishing experience.

## Seasonal Fishing Patterns

Seasonal fishing patterns are crucial for anglers to understand, as trout